In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.llms.groq import Groq
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner
import arxiv
import os 

In [ ]:
llm = Groq(
    model="llama-3.3-70b-versatile",
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [ ]:
def calcular_imposto_renda(rendimento: float) -> str:
    if rendimento <= 22847.76:
        return "Isento de Imposto de Renda"
    elif rendimento <= 33919.80:
        imposto = (rendimento - 22847.76) * 0.075
    elif rendimento <= 45012.60:
        imposto = (rendimento - 33919.80) * 0.15 + 826.29
    elif rendimento <= 55976.16:
        imposto = (rendimento - 45012.60) * 0.225 + 1428.14
    else:
        imposto = (rendimento - 55976.16) * 0.275 + 2203.91

    return f"Imposto de Renda devido: R$ {imposto:.2f}"

In [ ]:
ferramenta_calculo_imposto = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="calcular_imposto_renda",
    description=(
        "Calcula o Imposto de Renda devido com base no rendimento anual. "
        "Argumento: rendimento (float) - Rendimento anual em reais. "
        "Retorna uma mensagem indicando se é isento ou o valor do imposto a pagar."
    ),

)

In [ ]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    llm=llm,
    tools=[ferramenta_calculo_imposto],
    verbose=True,
    allow_parallel_tool_calls=True, # Permite chamadas paralelas de ferramentas
)

In [ ]:
agent_imposto = AgentRunner(agent_worker_imposto)


In [ ]:
response = agent_imposto.chat(
    "Qual é o imposto de renda devido para um rendimento anual de R$ 50000.00?",
)

In [ ]:
def consulta_artigos(title: str) -> str:
    """
    Consulta artigos na base dados ArXiv e retorna resultados formatados.
    """
    # O arqiv é uma base de dados de artigos científicos
    buscar = arxiv.Search(
        query=title,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance,
    )

    resultados = [
        f"Título: {artigo.title}\n"
        f"Categoria: {artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in buscar.results()
    ]

    return "\n\n".join(resultados) if resultados else "Nenhum artigo encontrado."

In [ ]:
consulta_artigos_tool = FunctionTool.from_defaults(
    fn=consulta_artigos,
    name="consulta_artigos",
    description=(
        "Consulta artigos na base de dados ArXiv com base no título fornecido. "
        "Argumento: title (str) - Título do artigo a ser pesquisado. "
        "Retorna uma lista formatada de artigos encontrados."
    ),
)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    llm=llm,
    tools=[ferramenta_calculo_imposto, consulta_artigos_tool],
    verbose=True,
    # Desabilita chamadas paralelas de ferramentas, pois eu quero sequencialmente
    allow_parallel_tool_calls=False,  
)

In [ ]:
agent = AgentRunner(agent_worker)
response2 = agent.chat(
    "Quais artigos existem sobre 'Machine Learning'?",
)

UTILIZANDO O TAVILY

In [ ]:
import os 
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
tavily_api_key = os.environ.get("TAVILY_API_KEY")


In [ ]:

from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(api_key=tavily_api_key) # type: ignore

In [ ]:
tavily_tool_list = tavily_tool.to_tool_list()

for tool in tavily_tool_list:
    print(tool.metadata.name)

In [ ]:
tavily_tool.search("Me retorne artigos sobre 'Machine Learning'", max_results=3) 

In [ ]:
tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="tavily_search",
    description=(
        "Busca artigos e informações na web usando a API Tavily. "
        "Argumento: query (str) - Termo de pesquisa. "
        "Retorna uma lista de resultados relevantes."
    ),
)

In [ ]:
agent_woker = FunctionCallingAgentWorker.from_tools(
    llm=llm,
    tools=[tavily_tool_function],
    verbose=True,
    allow_parallel_tool_calls=False,  
)

In [ ]:
agent = AgentRunner(agent_woker)
response3 = agent.chat(
    "Me retorne artigos sobre 'Machine Learning'",

)

In [ ]:
url = "files/LLM.pdf"
artigo = SimpleDirectoryReader(input_files=[url]).load_data() 

In [ ]:
url = "files/LLM_2.pdf"
tutorial = SimpleDirectoryReader(input_files=[url]).load_data() 

GERAR OS EMBEDDINGS

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = "intfloat/multilingual-e5-large"
)

artigo_index = VectorStoreIndex.from_documents(
    artigo,
    embed_model=Settings.embed_model,
)

tutorial_index = VectorStoreIndex.from_documents(
    tutorial,
    embed_model=Settings.embed_model,
)

In [ ]:
artigo_index.storage_context.persist(
    persist_dir="artigo",
)

tutorial_index.storage_context.persist(
    persist_dir="tutorial",
)

ENGINE DE BUSCA

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage
storagte_context = StorageContext.from_defaults(
    persist_dir="artigo",
)

artigo_index = load_index_from_storage(
    storage_context=storagte_context,
)

tutorial_storage_context = StorageContext.from_defaults(
    persist_dir="tutorial",
)

tutorial_index = load_index_from_storage(
    storage_context=tutorial_storage_context,
)

In [ ]:
artigo_engine = artigo_index.as_query_engine(
    similarity_top_k=3, llm=llm,
)

tutorial_engine = tutorial_index.as_query_engine(
    similarity_top_k=3, llm=llm,
)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description=(
                "Fornece informações sobre LLM e LangChain."
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta"
            )
        )
    ),
    QueryEngineTool(
        query_engine=tutorial_engine,
        metadata=ToolMetadata(
            name="tutorial_engine",
            description=(
                "Fornece informações sobre casos de uso e aplicações em LLMs."
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta"
            )
        )
    ),
]

In [8]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools, # type: ignore
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

agent_document = AgentRunner(agent_worker)

NameError: name 'FunctionCallingAgentWorker' is not defined

In [ ]:
response = agent_document.chat(
    "Quais as principais aplicações posso construir com LLM e LangChain?"
)

In [ ]:
response = agent_document.chat(
    "Quais são os casos de uso e aplicações de LLMs?",
)

AGENT ReAct

In [6]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    query_engine_tools,  # type: ignore
    verbose=True,
    llm=llm,
)

response = agent.chat(
    "Quais são os casos de uso e aplicações de LLMs?",
)

NameError: name 'query_engine_tools' is not defined